In [9]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))


/var/folders/lj/2t8twtsx5v1dw1dkbr59j1xr0000gn/T/ipykernel_71243/3522731578.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [10]:
import mysql.connector
from collections import defaultdict

# Database connection
conn = mysql.connector.connect(
    host='betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com',
    user='admin',
    password='7nRB1i2&A-K>',
    database='betting_db'
)

cursor = conn.cursor()

# Query to get championship payouts
query = """
    SELECT 
        legs.ParticipantName,
        legs.EventType,
        SUM(bets.PotentialPayout) AS total_payout
    FROM bets
    JOIN legs ON bets.WagerID = legs.WagerID
    WHERE 
        bets.LegCount = 1
        AND bets.WLCA = 'Active'
        AND bets.WhichBankroll = 'GreenAleph'
        AND legs.LeagueName = 'NFL'
        AND legs.EventType IN ('Semifinals Winner', 'Championship')
    GROUP BY legs.ParticipantName, legs.EventType;
"""

cursor.execute(query)

# Payouts dictionary
payouts = defaultdict(lambda: {'semifinal_payout': 0, 'championship_payout': 0})
for participant_name, event_type, total_payout in cursor.fetchall():
    normalized_name = participant_name.strip().lower()
    if event_type.strip().lower() == "semifinals winner":
        payouts[normalized_name]['semifinal_payout'] = float(total_payout)
    elif event_type.strip().lower() == "championship":
        payouts[normalized_name]['championship_payout'] = float(total_payout)

cursor.close()
conn.close()

# Teams and probabilities
teams = {
    "kansas city chiefs": {"current_round_prob": 0.7},
    "buffalo bills": {"current_round_prob": 0.3},
    "baltimore ravens": {"current_round_prob": 0.6},
    "houston texans": {"current_round_prob": 0.4},
    "detroit lions": {"current_round_prob": 0.5},
    "philadelphia eagles": {"current_round_prob": 0.5},
    "los angeles rams": {"current_round_prob": 0.5},
    "washington commanders": {"current_round_prob": 0.5}
}

# Matchups
matchups = {
    "quarterfinals": [
        ("kansas city chiefs", "houston texans"),
        ("buffalo bills", "baltimore ravens"),
        ("detroit lions", "washington commanders"),
        ("philadelphia eagles", "los angeles rams")
    ]
}

# Hypothetical probabilities
hypothetical_probabilities = {
    "kansas city chiefs": {"houston texans": 0.6, "buffalo bills": 0.55, "baltimore ravens": 0.65, "washington commanders": 0.5},
    "buffalo bills": {"baltimore ravens": 0.5, "houston texans": 0.6, "kansas city chiefs": 0.45, "los angeles rams": 0.55},
    "baltimore ravens": {"buffalo bills": 0.55, "houston texans": 0.5, "kansas city chiefs": 0.4, "philadelphia eagles": 0.5},
    "houston texans": {"kansas city chiefs": 0.4, "buffalo bills": 0.6, "baltimore ravens": 0.45, "detroit lions": 0.5},
    "detroit lions": {"washington commanders": 0.35, "philadelphia eagles": 0.6, "los angeles rams": 0.5, "baltimore ravens": 0.55},
    "philadelphia eagles": {"los angeles rams": 0.5, "washington commanders": 0.4, "detroit lions": 0.6, "kansas city chiefs": 0.55},
    "los angeles rams": {"philadelphia eagles": 0.55, "washington commanders": 0.5, "detroit lions": 0.5, "buffalo bills": 0.4},
    "washington commanders": {"detroit lions": 0.4, "philadelphia eagles": 0.45, "los angeles rams": 0.5, "kansas city chiefs": 0.6}
}

# Calculate Portfolio EV
def calculate_portfolio_ev(winning_team, losing_team):
    portfolio_ev = 0

    # Semifinal EV for the winning team
    semifinal_ev = (
        teams[winning_team]['current_round_prob'] *
        payouts[winning_team]['semifinal_payout']
    )

    # Semifinal EV for the other conference
    other_conference_ev = 0
    other_conference_teams = ["detroit lions", "philadelphia eagles", "los angeles rams", "washington commanders"] if winning_team in ["kansas city chiefs", "buffalo bills", "baltimore ravens", "houston texans"] else ["kansas city chiefs", "buffalo bills", "baltimore ravens", "houston texans"]

    for team, data in teams.items():
        if team in other_conference_teams:
            for opponent, prob in hypothetical_probabilities[team].items():
                if opponent in other_conference_teams:
                    other_conference_ev += (
                        data['current_round_prob'] *
                        prob * payouts[team]['semifinal_payout']
                    )

    # Championship EV for the winning team
    championship_ev = 0
    for opponent in other_conference_teams:
        opponent_prob_reaches_championship = (
            teams[opponent]['current_round_prob'] *
            max(hypothetical_probabilities[opponent].values())
        )
        prob_team_beats_opponent = hypothetical_probabilities[winning_team].get(opponent, 0.5)
        prob_opponent_beats_team = 1 - prob_team_beats_opponent

        championship_ev += (
            teams[winning_team]['current_round_prob'] *
            opponent_prob_reaches_championship *
            (
                prob_team_beats_opponent * payouts[winning_team]['championship_payout'] +
                prob_opponent_beats_team * payouts[opponent]['championship_payout']
            )
        )

    # Contribution from other advancing teams in the same conference
    other_team_championship_ev = 0
    same_conference_teams = ["kansas city chiefs", "buffalo bills", "baltimore ravens", "houston texans"] if winning_team in ["kansas city chiefs", "buffalo bills", "baltimore ravens", "houston texans"] else ["detroit lions", "philadelphia eagles", "los angeles rams", "washington commanders"]

    for t1 in same_conference_teams:
        if t1 == winning_team or t1 == losing_team:
            continue

        prob_t1_reaches_championship = (
            teams[t1]['current_round_prob'] *
            hypothetical_probabilities[t1].get(winning_team, 0.5)  # T1 beats the winning team
        )

        for t2 in other_conference_teams:
            prob_t2_reaches_championship = (
                teams[t2]['current_round_prob'] *
                max(hypothetical_probabilities[t2].values())
            )

            prob_t1_beats_t2 = hypothetical_probabilities[t1].get(t2, 0.5)
            prob_t2_beats_t1 = 1 - prob_t1_beats_t2

            matchup_ev = (
                prob_t1_reaches_championship *
                prob_t2_reaches_championship *
                (
                    prob_t1_beats_t2 * payouts[t1]['championship_payout'] +
                    prob_t2_beats_t1 * payouts[t2]['championship_payout']
                )
            )

            other_team_championship_ev += matchup_ev

    # Total EV
    portfolio_ev = semifinal_ev + other_conference_ev + championship_ev + other_team_championship_ev
    return portfolio_ev

# EV for all scenarios
results = []
for matchup in matchups["quarterfinals"]:
    team1, team2 = matchup
    ev_team1_wins = calculate_portfolio_ev(team1, team2)
    ev_team2_wins = calculate_portfolio_ev(team2, team1)
    results.append({
    "Scenario": f"{team1.title()} beats {team2.title()}",
    "Portfolio EV": ev_team1_wins
    })
    results.append({
        "Scenario": f"{team2.title()} beats {team1.title()}",
        "Portfolio EV": ev_team2_wins
    })

# Output the results
for result in results:
    print(f"Scenario: {result['Scenario']}, Portfolio EV: ${result['Portfolio EV']:.2f}")



Scenario: Kansas City Chiefs beats Houston Texans, Portfolio EV: $346630.71
Scenario: Houston Texans beats Kansas City Chiefs, Portfolio EV: $260770.38
Scenario: Buffalo Bills beats Baltimore Ravens, Portfolio EV: $295287.66
Scenario: Baltimore Ravens beats Buffalo Bills, Portfolio EV: $380682.33
Scenario: Detroit Lions beats Washington Commanders, Portfolio EV: $317384.03
Scenario: Washington Commanders beats Detroit Lions, Portfolio EV: $333032.82
Scenario: Philadelphia Eagles beats Los Angeles Rams, Portfolio EV: $242440.39
Scenario: Los Angeles Rams beats Philadelphia Eagles, Portfolio EV: $414436.57
